In [5]:
import numpy as np
import pyomo.environ as pyEnv

In [29]:
# 2 areas, expected profits for 4 bicycles, 2 categories
areas = [
    [[1, 5], [2, 3], [3, 2], [4, 1]],
    [[8, 3], [5, 4], [2, 7], [1, 6]]
]
matrix = np.array(areas)
a = [5, 10] # surplus of bicycles available
s = [1, 2] # space each bicycle category occupies
T = 50 # total space available

In [30]:
from pyomo.environ import *

# Definir dados do problema
areas = [
    [[1, 5], [2, 3], [3, 2], [4, 1]],
    [[8, 3], [5, 4], [2, 7], [1, 6]]
]
matrix = np.array(areas)
a = [5, 10]  # excedente de bicicletas disponíveis
s = [1, 2]   # espaço ocupado por cada categoria de bicicleta
T = 50      # capacidade total do caminhão

model = ConcreteModel()

# Índices para as áreas e categorias
model.I = Set(initialize=range(matrix.shape[0]))
model.J = Set(initialize=range(matrix.shape[2]))

# Variáveis de decisão
model.x = Var(model.I, model.J, domain=NonNegativeIntegers)

# Função objetivo
model.profit = Objective(
    expr=sum(matrix[i, k, j] * model.x[i, j] for i in model.I for j in model.J for k in range(matrix.shape[1])),
    sense=maximize
)

# Restrições
def supply_constraint(model, j):
    return sum(model.x[i, j] for i in model.I) <= a[j]

model.supply_constraint = Constraint(model.J, rule=supply_constraint)

def capacity_constraint(model):
    return sum(s[j] * model.x[i, j] for i in model.I for j in model.J) <= T

model.capacity_constraint = Constraint(rule=capacity_constraint)

# Resolver o modelo
solver = SolverFactory('glpk')
results = solver.solve(model)

# Mostrar resultados
print("Status:", results.solver.termination_condition)
print("Valor objetivo ótimo (soma dos lucros esperados):", value(model.profit))

print("\nSolução ótima:")
for i in model.I:
    for j in model.J:
        print(f"Número de bicicletas da categoria {j + 1} a serem movidas para a área {i + 1}: {model.x[i, j].value}")


cbc


ApplicationError: No executable found for solver 'cbc'